### Cheking if we are in google colab or not.

In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
IN_COLAB

True

In [ ]:
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%capture
if IN_COLAB:
    !pip install geemap
    !pip install geedim

In [ ]:
import ee
import geemap

In [ ]:
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
from skimage import io
import numpy as np
import logging
import multiprocessing
import os
import requests
import shutil
# from retry import retry
from datetime import datetime
import matplotlib.pyplot as plt

from collections import namedtuple

In [ ]:
import sys
if IN_COLAB:
    sys.path.insert(0, '/content/drive/MyDrive/TemporalGAN-main/dataset')

In [ ]:
from utils.utils import *
from utils.ee_utils import *
from utils.ee_utils import TextColors as tc

4/1AfgeXvvmWF-_z0qOvwsZK597gUnHMOdSTjpY6CStI-buWxqNvRlGoRU19og

In [ ]:
Map = geemap.Map()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=-lKPYOVu7lg803LsRfg0BRDy4g51P5ijGPRmMiEIKZc&tc=xtT1pBCfc4Y2IVfbmylp9c0w5xtKOaCctiXHfGRc4J4&cc=vjY3g2_6AncYaDj4W7S8zmCNCVVqWOh2Bx9h6E7TJA4

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AVHEtk7b4OmoODTVPi7uTTJDuIIS4LyIWFLZ1Q7GzLSxknrrq9LP_USIddQ

Successfully saved authorization token.


In [ ]:
# ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
ee.Initialize()

# A Function to retrive S1 and S2

has two inputs:
*   date
*   roi


Returns:
*   s1
*   s2 image




In [ ]:
from datetime import date, datetime
from dateutil.relativedelta import relativedelta

---

# OSCD Dataset ROIs

### Sentinel 2 Surface Reflectance band 10
band 10 is cirus and is available in TOA data, and is used for atmospheric correction, therfore it is not in the SR img.

# Downlaod All!

In [ ]:
import pandas as pd
import ast
import collections
import os
import shutil

In [ ]:
YEAR = 2017
Y = YEAR - 2000
if IN_COLAB:
  rois_path = f'/content/drive/MyDrive/TemporalGAN-main/dataset//{Y}_ROIs.xlsx'
else:
  rois_path = f'D:\python\TemporalGAN\dataset\\{Y}_ROIs.xlsx'
SR = False if YEAR == 2017 else True

In [ ]:
df = pd.read_excel(rois_path)

## checking if there is a duplicate in the names

In [ ]:
names = []
for index, row in df.iterrows():
    names.append(row['name']+str(row['year_type']))
print([item for item, count in collections.Counter(names).items() if count > 1])

[]


Remove unwanted folders in `colab`

In [ ]:
#shutil.rmtree('/content/2021')  

In [ ]:
# Consice info printing function
sen2_short_print = lambda s2_collection: ee_property_printer(s2_collection, propertie_name_list=['system:time_start','roi_cloud_cover', 'CLOUDY_PIXEL_PERCENTAGE',
                                                                                                            'system:band_names'],
                                                             first_is_t_in_millis=True, df_instead_of_print=False)

sen1_short_print = lambda s2_collection: ee_property_printer(s2_collection, propertie_name_list=['system:time_start','platform_number',
                                                                                           'system:band_names','instrumentMode',                         
                                                                                            'relativeOrbitNumber_stop','relativeOrbitNumber_start',
                                                                                            'cycleNumber'],
                                                             first_is_t_in_millis=True, df_instead_of_print=False)

### dataset downlaoder

In [ ]:
import os

current_dir = os.getcwd()

print(current_dir)

/content


In [ ]:
if IN_COLAB:
    CURRENT_DIR = '/content/drive/MyDrive/TemporalGAN-main/dataset/s1s2/'
    S1_FOLDER_TRAIN = f'{CURRENT_DIR}{YEAR}/s1_imgs/train/'
    S1_FOLDER_TEST = f'{CURRENT_DIR}{YEAR}/s1_imgs/test/'
    S2_FOLDER_TRAIN = f'{CURRENT_DIR}{YEAR}/s2_imgs/train/'
    S2_FOLDER_TEST = f'{CURRENT_DIR}{YEAR}/s2_imgs/test/'
else:
    S1_FOLDER_TRAIN = f'./s1s2/{YEAR}/s1_imgs/train/'
    S1_FOLDER_TEST = f'./s1s2/{YEAR}/s1_imgs/test/'
    S2_FOLDER_TRAIN = f'./s1s2/{YEAR}/s2_imgs/train/'
    S2_FOLDER_TEST = f'./s1s2/{YEAR}/s2_imgs/test/'
# create folders if not exist
if not os.path.exists(S1_FOLDER_TRAIN) or not os.path.exists(S1_FOLDER_TEST) or not os.path.exists(S2_FOLDER_TRAIN) or not os.path.exists(S2_FOLDER_TEST):
    print("Creating folders...")
    os.makedirs(S1_FOLDER_TRAIN, exist_ok=True)
    os.makedirs(S1_FOLDER_TEST, exist_ok=True)
    os.makedirs(S2_FOLDER_TRAIN, exist_ok=True)
    os.makedirs(S2_FOLDER_TEST, exist_ok=True)
    print('DONE!')

Creating folders...
DONE!


In [ ]:
# df = df.iloc[14:]

In [ ]:
bands = ['B2','B3','B4','B8','B11','B12']
for index, row in df.iterrows():
    print("🟢",tc.BOLD_BAKGROUNDs.PURPLE,row['name']+str(row['year_type']),tc.ENDC,"🟢")
    r_roi = ast.literal_eval(row['roi'])
    r_roi = ee.Geometry.Polygon(r_roi, None, False)
    r_date = ast.literal_eval(row['date'])
    r_pp =  row['priority_path']
    r_mc =  float(row['max_cloud'])
    r_msn =  float(row['max_snow'])
    r_ms =  int(row['month_span'])
    r_rd =  int(row['retry_days'])

    s2_col,s1_col = s1s2(r_roi , date = r_date , priority_path = r_pp , max_cloud =r_mc , max_snow = r_msn , month_span = r_ms ,retry_days = r_rd , sr = SR)
    s1_col = s1_col.map(toLinear) # convers dB values to Linear, and adds band 'VV_lin' to the images of the collection
    print(tc.OKCYAN,"---------------------------S2---------------------------",tc.ENDC)
    sen2_short_print(s2_col)
    print(tc.OKCYAN,"---------------------------S1---------------------------",tc.ENDC)
    sen1_short_print(s1_col)
    
    #=====================DOWNLOAD==========================
    s1_img = s1_col.mean().clip(r_roi)
    s1_img = toDb(s1_img) # reads the 'VV_lin' and returns the image with the added band 'VV_db' 
    
    if s2_col.size().getInfo()< 3:
        s2_img = s2_col.sort('roi_cloud_cover').first().clip(r_roi).multiply(0.0001)
    else:    
        s2_img = s2_col.median().clip(r_roi).multiply(0.0001)

    s1_img_vv = s1_img.select('VV_db')
    s2_img_12b = s2_img.select(bands)
    
    S1_FOLDER = S1_FOLDER_TRAIN if row['train_test'] == 'train' else S1_FOLDER_TEST
    S2_FOLDER = S2_FOLDER_TRAIN if row['train_test'] == 'train' else S2_FOLDER_TEST
        
    r_name = str(index+1).zfill(3) + '_' + row['name'] + str(row['year_type'])
    geemap.download_ee_image(s1_img_vv, S1_FOLDER +r_name+".tif",crs='EPSG:4326', scale=10,region = r_roi)
    geemap.download_ee_image(s2_img_12b, S2_FOLDER +r_name+".tif",crs='EPSG:4326', scale=10,region = r_roi)

🟢  abudhabi17  🟢
 ◍◍Finding S2 
 ◍Single scene coverege was found! 
 ◍◍Finding S1 
mean s2 date:  2017-06-19
final date range:   2017-04-29 2017-08-08 | diff_days -> 101 
◍checking for DESCENDING single scene
Mosiac Covers 96.63861773206382 percent of the roi.
Mosaic Coverege Accepted
Mosaic Coverege Accepted
 ◍Collection Found! 
 ---------------------------S2--------------------------- 
Collection Size:  3
system:time_start-------> ['2017-06-07', '2017-06-17', '2017-07-02']
roi_cloud_cover---------> [0.04678153025371515, 0.03782336488598246, 0]
CLOUDY_PIXEL_PERCENTAGE-> [0.1071, 0.0681, 0]
system:band_names-------> [['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'QA10', 'QA20', 'QA60'], ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'QA10', 'QA20', 'QA60'], ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'QA10', 'QA20', 'QA60']]
 ---------------------------S1---------------------

001_abudhabi17.tif: |          | 0.00/2.07M (raw) [  0.0%] in 00:00 (eta:     ?)

001_abudhabi17.tif: |          | 0.00/12.4M (raw) [  0.0%] in 00:00 (eta:     ?)

🟢  abudhabi_buffed17  🟢
 ◍◍Finding S2 
 ◍Single scene coverege was found! 
 ◍◍Finding S1 
mean s2 date:  2017-06-19
final date range:   2017-04-29 2017-08-08 | diff_days -> 101 
◍checking for DESCENDING single scene
Mosiac Covers 98.28746546491728 percent of the roi.
Mosaic Coverege Accepted
Mosaic Coverege Accepted
 ◍Collection Found! 
 ---------------------------S2--------------------------- 
Collection Size:  3
system:time_start-------> ['2017-06-07', '2017-06-17', '2017-07-02']
roi_cloud_cover---------> [0.04678153025371515, 0.03782336488598246, 0]
CLOUDY_PIXEL_PERCENTAGE-> [0.1071, 0.0681, 0]
system:band_names-------> [['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'QA10', 'QA20', 'QA60'], ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'QA10', 'QA20', 'QA60'], ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'QA10', 'QA20', 'QA60']]
 ---------------------------S1--------------

002_abudhabi_buffed17.tif: |          | 0.00/35.5M (raw) [  0.0%] in 00:00 (eta:     ?)

...

043_valencia17.tif: |          | 0.00/2.29M (raw) [  0.0%] in 00:00 (eta:     ?)

043_valencia17.tif: |          | 0.00/13.8M (raw) [  0.0%] in 00:00 (eta:     ?)

🟢  valencia_buffed17  🟢
 ◍◍Finding S2 
 ◍Single scene coverege was found! 
 ◍◍Finding S1 
mean s2 date:  2017-06-27
final date range:   2017-05-27 2017-07-27 | diff_days -> 61 
◍checking for DESCENDING single scene
Mosiac Covers 98.85507878646625 percent of the roi.
Mosaic Coverege Accepted
Mosaic Coverege Accepted
 ◍Collection Found! 
 ---------------------------S2--------------------------- 
Collection Size:  3
system:time_start-------> ['2017-06-15', '2017-06-30', '2017-07-05']
roi_cloud_cover---------> [0, 0.9493166910527835, 0.26111393127428245]
CLOUDY_PIXEL_PERCENTAGE-> [0, 0.824, 0.231]
system:band_names-------> [['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'QA10', 'QA20', 'QA60'], ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'QA10', 'QA20', 'QA60'], ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'QA10', 'QA20', 'QA60']]
 ---------------------------S1------------------

044_valencia_buffed17.tif: |          | 0.00/37.3M (raw) [  0.0%] in 00:00 (eta:     ?)

044_valencia_buffed17.tif: |          | 0.00/224M (raw) [  0.0%] in 00:00 (eta:     ?)

In [ ]:
if IN_COLAB:
    ! du -sh /content/drive/MyDrive/TemporalGAN-main/dataset/s1s2/2021
    ! du -sh /content/drive/MyDrive/TemporalGAN-main/dataset/s1s2/2019
    ! du -sh /content/drive/MyDrive/TemporalGAN-main/dataset/s1s2/2017

2.3G	/content/drive/MyDrive/TemporalGAN-main/dataset/s1s2/2021
2.3G	/content/drive/MyDrive/TemporalGAN-main/dataset/s1s2/2019
2.3G	/content/drive/MyDrive/TemporalGAN-main/dataset/s1s2/2017


## Saving downloaded dataset for later use in drive

In [ ]:
# from distutils.dir_util import copy_tree
# copy_tree("/content/s1_imgs", "/content/drive/MyDrive/Colab Notebooks/rgb2sar/dataset/s1")
# copy_tree("/content/s2_imgs", "/content/drive/MyDrive/Colab Notebooks/rgb2sar/dataset/s2")

# patching the files

## importing patch function
`perfect_patchify` is what I worte on my own.

we also install and import `patchify` library

In [ ]:
if IN_COLAB:
    !pip install patchify

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import sys
if IN_COLAB:
    sys.path.insert(0,'/content/drive/MyDrive/Colab Notebooks/rgb2sar/pyModules/')
    from utils.utils import perfect_patchify  

In [ ]:
if IN_COLAB:
    from patchify import patchify

##loading the data from drive to working directory

In [ ]:
# from distutils.dir_util import copy_tree
# copy_tree("/content/drive/MyDrive/Colab Notebooks/rgb2sar/dataset/s1","/content/s1_imgs")
# copy_tree("/content/drive/MyDrive/Colab Notebooks/rgb2sar/dataset/s2","/content/s2_imgs")

KeyboardInterrupt: ignored

In [ ]:
from skimage import io

## patchify s2 images

In [ ]:
# shutil.rmtree('/content/s2_imgs_patched//')

In [ ]:
if not os.path.exists('/content/s1s2') and IN_COLAB:
    os.mkdir('/content/s1s2')

In [ ]:
if IN_COLAB:    
    if not os.path.exists('/content/s1s2/s1_imgs_patched//'):
        os.mkdir('/content/s1s2/s1_imgs_patched//')
    if not os.path.exists('/content/s1s2/s2_imgs_patched//'):
        os.mkdir('/content/s1s2/s2_imgs_patched//')

In [ ]:
path = '/content/s2_imgs//'
opath = '/content/s1s2/s2_imgs_patched//'
files = os.listdir(path)
files.sort()

for file in files:
    print('📍📍📍🗺️ \033[92m', file ,'\033[0m 🗺️📍📍📍')
    x=io.imread(path+file)
    img = reshape_array(x)
    print(img.shape)



    print("range before norm: ",np.min(img),np.mean(img),np.std(img),np.max(img))

    print(img.shape) #sanity check to see if masks are the same size as images
    
    img = add_geo_context(img,file,geo_context)

    

    img[img>0.99] = 0.99
    img[img<0] = 0
    img = nan_remover(img)

    print("range after norm and NaN removal: ",np.min(img),np.mean(img),np.std(img),np.max(img))

    if ('adj' in file) or ('buffed' in file):
      print('🟥not main')
      patches = perfect_patchify(img,mute=True)
    else:
      print('🟦main')
      patches = patchify(img,(256,256,13),step=128)
      patches = patches[:,:,0,:,:,:]
    
    print('✅',patches.shape)

    # SAVING PATHCES
    img_index = file[0:3]
    for i in range(patches.shape[0]):
        for j in range(patches.shape[1]):
            patch = patches[i,j,:,:,:]
            #print(patch.shape)
            patch = np.swapaxes(patch, 2,0)
            patch= np.swapaxes(patch, 1,2)
            
            io.imsave(opath +img_index+ '_r'+ str(i).zfill(2) + '_c' + str(j).zfill(2) + '.tif', patch)

## checking the NaN ratio

In [ ]:
! du -sh /content/s1s2/s2_imgs_patched//

In [ ]:
patched_s2 = os.listdir('/content/s1s2/s2_imgs_patched//')
print(len(patched_s2))

In [ ]:
# s2_test = io.imread('/content/s2_imgs/013_beirut20.tif')

# p=s2_test
# print(p.shape)
# if (len(p.shape)==3):
#     if p.shape[0]<p.shape[2] :
#         q = np.swapaxes(p, 2,0)
#         s2_test = np.swapaxes(q, 1,0)
#     else:
#         s2_test = p
# else:
#     s2_test = np.expand_dims(p,axis=2)


# print(s2_test.shape)

# print(np.min(s2_test),np.mean(s2_test),np.max(s2_test))
# print(np.count_nonzero(np.isnan(s2_test)))
# s2_test[np.isnan(s2_test)] = 0
# print(np.min(s2_test),np.mean(s2_test),np.max(s2_test))
# # s2_test[np.isnan(s2_test)] = 100
# # s2_test[~np.isnan(s2_test)] = 0
# plt.imshow(s2_test[:,:,[3,2,1]])
# plt.colorbar()


In [ ]:
path = '/content/s1_imgs//'
opath = '/content/s1s2/s1_imgs_patched//'
files = os.listdir(path)
files.sort()

for file in files:
    print('📍📍📍🗺️ \033[92m', file ,'\033[0m 🗺️📍📍📍')
    x=io.imread(path+file)
    img = reshape_array(x, channel_first = False)
    print(img.shape)



    print("range before norm: ",np.min(img),np.mean(img),np.std(img),np.max(img))

    print(img.shape) #sanity check to see if masks are the same size as images
    
    #img = add_geo_context(img,file,geo_context)

    

    img[img>15] = 15
    img[img<-25] = -25
    img = nan_remover(img)

    print("range after norm and NaN removal: ",np.min(img),'\033[96m',np.mean(img),np.std(img),'\033[0m',np.max(img))

    if ('adj' in file) or ('buffed' in file):
      print('🟥not main')
      patches = perfect_patchify(img,mute=True)
    else:
      print('🟦main')
      patches = patchify(img,(256,256,1),step=128)
      patches = patches[:,:,0,:,:,:]
    
    print('✅',patches.shape)

    # SAVING PATHCES
    img_index = file[0:3]
    for i in range(patches.shape[0]):
        for j in range(patches.shape[1]):
            patch = patches[i,j,:,:,:]
            #print(patch.shape)
            # patch = np.swapaxes(patch, 2,0)
            # patch= np.swapaxes(patch, 1,2)
            
            io.imsave(opath +img_index+ '_r'+ str(i).zfill(2) + '_c' + str(j).zfill(2) + '.tif', patch)

## checking if the dimentions are  correct

In [ ]:
x=io.imread('/content/s1s2/s1_imgs_patched/000_r00_c01.tif')
# img = shape_correction(x)
print(x.shape)
plt.imshow(x[:,:,0])
# plt.imshow(x[3,:,:])

In [ ]:
x=io.imread('/content/s1s2/s2_imgs_patched/000_r00_c01.tif')
# img = shape_correction(x)
print(x.shape)
# plt.imshow(x[:,:,0])
plt.imshow(x[2,:,:])

In [ ]:
! du -sh /content/s1s2/s1_imgs_patched//

In [ ]:
patched_s1 = os.listdir('/content/s1s2/s1_imgs_patched//')
print(len(patched_s1))

## converting the patched folders into zip files

In [ ]:
!zip -r '/content/s1s2.zip' '/content/s1s2'

In [ ]:
! du -sh /content/s1s2.zip

In [ ]:
# from IPython.display import FileLink
# FileLink(r's1s2.zip')

In [ ]:
!cp "/content/s1s2.zip" "/content/drive/MyDrive/Colab Notebooks/rgb2sar/dataset/"